<a href="https://colab.research.google.com/github/tinemyumi/saude-mental-datasus/blob/main/codigos-python/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Relatório**

**Integrantes**
- Larissa Yumi Tinem
- Leo Tsuchida Hoçoya

**Conteúdo**
- Código para download dos dados do datasus e IBGE, além da transferência permamente desses dados para o Google Drive.

**Histórico de alterações:**
- **13/09/2025** - Larissa Tinem - Criação do arquivo e importação dos dados do SIH-SUS.
- **20/09/2025** - Larissa Tinem - Obtenção dos dados do IBGE usando a API do IBGE via pysus (IBGE.FetchData)



# **Baixando dados do Datasus (SIH-SUS) e IBGE via pysus**

Baixando dados do Datasus(SIH-SUS, Estado de São Paulo, no ano de 2015) através da biblioteca pysus.

In [1]:
# Instalação da biblioteca pysus
!pip install pysus

In [ ]:
# Código utilizado para realizar o download da base da de dados do datasus, a respeito do SIH-SUS

from pysus import SIH
import os
import pandas as pd

# 1. Carregando a biblioteca PySUS, os arquivos do datasus
sih = SIH().load()

# 2. Definição dos anos para realizar download (como exemplo o ano de 2015, mas pode ser uma lista dos anos desejáveis)
anos_para_baixar = [2015]
diretorio_base = '/content/dados_sih/dados_datasus/'

# 3. Loop para processar cada ano
for ano in anos_para_baixar:
    diretorio_ano = os.path.join(diretorio_base, str(ano))
    os.makedirs(diretorio_ano, exist_ok=True)

    print(f"Buscando e baixando dados para o ano: {ano}...")

    # Buscando os arquivos para o ano atual e para todos os meses
    arquivos_ano = sih.get_files("RD", uf="sp", year=ano, month=list(range(1, 13)))

    # Baixando os arquivos para a pasta específica do ano
    sih.download(arquivos_ano, local_dir=diretorio_ano)

    print(f"Download para {ano} concluído na pasta: {diretorio_ano}")

print("\nProcesso de download para todos os anos concluído!")

In [ ]:
# Código de importação dados do content para o Google Drive

from google.colab import drive
import shutil

# Monta o seu Google Drive no Colab
drive.mount('/content/drive')

# Caminho da pasta de origem no Colab
caminho_origem = '/content/dados_sih/'

# Caminho de destino Google Drive
# A pasta principal do seu Drive é '/content/drive/MyDrive/'
caminho_destino = '/content/drive/MyDrive/Dado DataSUS/dados_sih/'

print("Iniciando a transferência dos arquivos para o Google Drive...")

try:
    shutil.move(caminho_origem, caminho_destino)
    print("\nTransferência concluída com sucesso!")
    print(f"Os arquivos agora estão salvos em: {caminho_destino}")

except FileNotFoundError:
    print("\nErro: A pasta de origem não foi encontrada. Verifique se o caminho está correto.")

except Exception as e:
    print(f"\nOcorreu um erro durante a transferência: {e}")


# **Obtendo dados do IBGE**

Obtendo dados do IBGE usando a API do IBGE via pysus (IBGE.FetchData)

Data de realização: 20/09
Autor: Larissa Yumi Tinem


In [2]:
import pandas as pd
from pysus.online_data import IBGE
from pysus.ftp.utils import zfill_year
import ipywidgets as widgets
%matplotlib inline

In [11]:
# Verificando os dados disponíveis por área de assunto, antes de realizar o download.

ag = IBGE.list_agregados()
ag

Fetching Data groupings from https://servicodados.ibge.gov.br/api/v3/agregados?


,id,nome,agregados
0,D5,Áreas Urbanizadas,"[{'id': '8418', 'nome': 'Áreas urbanizadas, Lo..."
1,CL,Cadastro Central de Empresas,"[{'id': '1685', 'nome': 'Unidades locais, empr..."
2,CA,Censo Agropecuário,"[{'id': '1005', 'nome': 'Número de estabelecim..."
3,ME,"Censo Comum do Mercosul, Bolívia e Chile","[{'id': '1221', 'nome': 'População residente, ..."
4,CD,Censo Demográfico,"[{'id': '10049', 'nome': 'Moradores indígenas ..."
...,...,...,...
64,VS,Produção da Extração Vegetal e da Silvicultura,"[{'id': '289', 'nome': 'Quantidade produzida e..."
65,PO,Produção de Ovos de Galinha,"[{'id': '6672', 'nome': 'Quantidade de ovos pr..."
66,IO,Produto Interno Bruto dos Municípios,"[{'id': '21', 'nome': 'Produto interno bruto a..."
67,XE,Projeção da População,"[{'id': '7358', 'nome': 'População, por sexo e..."


In [12]:
# Conjunto de dados agregados disponíveis dentro da categoria "Censo Demográfico"

pd.DataFrame(ag[ag.id=='CM'].agregados.iloc[0])

,id,nome
0,305,População residente em domicílios particulares...
1,319,Média de moradores por domicílio particular pe...
2,472,"População residente por idade, forma de declar..."
3,473,Pessoas de 4 anos ou mais de idade por grupos ...
4,475,"População residente por grupos de idade, sexo ..."
5,476,Pessoas de 4 anos ou mais de idade por frequên...
6,477,Pessoas de 4 anos ou mais que frequentam escol...
7,478,Pessoas não residentes no município de residên...
8,479,Chefes de domicílios particulares permanentes ...
9,480,População residente por relação com o chefe do...


In [49]:
# Consulta dados do IBGE via API em formato JSON

ds_sp = IBGE.FetchData(
    6579,
    periodos=2015,
    localidades='N6[all]',  # todos os municípios de SP
    view='flat'
)
ds.JSON

Fetching https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2015/variaveis/allxp?localidades=N6[all]&view=flat


[{'NC': 'Nível Territorial (Código)',
  'NN': 'Nível Territorial',
  'MC': 'Unidade de Medida (Código)',
  'MN': 'Unidade de Medida',
  'V': 'Valor',
  'D1C': 'Unidade da Federação (Código)',
  'D1N': 'Unidade da Federação',
  'D2C': 'Ano (Código)',
  'D2N': 'Ano',
  'D3C': 'Variável (Código)',
  'D3N': 'Variável'},
 {'NC': '3',
  'NN': 'Unidade da Federação',
  'MC': '45',
  'MN': 'Pessoas',
  'V': '1768204',
  'D1C': '11',
  'D1N': 'Rondônia',
  'D2C': '2015',
  'D2N': '2015',
  'D3C': '9324',
  'D3N': 'População residente estimada'},
 {'NC': '3',
  'NN': 'Unidade da Federação',
  'MC': '45',
  'MN': 'Pessoas',
  'V': '803513',
  'D1C': '12',
  'D1N': 'Acre',
  'D2C': '2015',
  'D2N': '2015',
  'D3C': '9324',
  'D3N': 'População residente estimada'},
 {'NC': '3',
  'NN': 'Unidade da Federação',
  'MC': '45',
  'MN': 'Pessoas',
  'V': '3938336',
  'D1C': '13',
  'D1N': 'Amazonas',
  'D2C': '2015',
  'D2N': '2015',
  'D3C': '9324',
  'D3N': 'População residente estimada'},
 {'NC': '3',

In [50]:
ds_sp.to_dataframe()

,NC,NN,MC,MN,V,D1C,D1N,D2C,D2N,D3C,D3N
0,Nível Territorial (Código),Nível Territorial,Unidade de Medida (Código),Unidade de Medida,Valor,Município (Código),Município,Ano (Código),Ano,Variável (Código),Variável
1,6,Município,45,Pessoas,25578,1100015,Alta Floresta D'Oeste - RO,2015,2015,9324,População residente estimada
2,6,Município,45,Pessoas,104401,1100023,Ariquemes - RO,2015,2015,9324,População residente estimada
3,6,Município,45,Pessoas,6355,1100031,Cabixi - RO,2015,2015,9324,População residente estimada
4,6,Município,45,Pessoas,87226,1100049,Cacoal - RO,2015,2015,9324,População residente estimada
...,...,...,...,...,...,...,...,...,...,...,...
5567,6,Município,45,Pessoas,13456,5222005,Vianópolis - GO,2015,2015,9324,População residente estimada
5568,6,Município,45,Pessoas,8171,5222054,Vicentinópolis - GO,2015,2015,9324,População residente estimada
5569,6,Município,45,Pessoas,5495,5222203,Vila Boa - GO,2015,2015,9324,População residente estimada
5570,6,Município,45,Pessoas,5578,5222302,Vila Propício - GO,2015,2015,9324,População residente estimada


In [51]:
# Salvando os dados como parquet no Google Drive

from google.colab import drive
drive.mount('/content/drive')

# Salvando como parquet
ds.to_dataframe().to_parquet('/content/drive/MyDrive/DadoDataSUS/dados_ibge/populacao_residente_2015.parquet', index=False)
ds_sp.to_dataframe().to_parquet('/content/drive/MyDrive/DadoDataSUS/dados_ibge/populacao_residente_municipio_2015.parquet', index=False)
print("Salvo com sucesso")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Salvo com sucesso
